In [59]:
import pandas as pd

In [60]:
df = pd.read_csv('0. merged_df.csv')

In [61]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [62]:
lon_min, lon_max = 11.5, 13
lat_min, lat_max = 45, 46

In [63]:
df = df.loc[(df.lon >= lon_min) & (df.lon <= lon_max)]
df = df.loc[(df.lat >= lat_min) & (df.lat <= lat_max)]
df

,osm_id,code,fclass,name,lon,lat,SELECT
0,4710535,7218,grass,NaN,12.272857,45.503580,nature
1,4710570,7218,grass,NaN,12.279067,45.505727,nature
2,4710648,7206,cemetery,Cimitero di Favaro Veneto,12.280370,45.511354,nature
3,4851260,7213,nature_reserve,Caserma Edmondo Matter,12.240170,45.505275,nature
4,4851276,7206,cemetery,Cimitero di Mestre,12.243382,45.501909,nature
...,...,...,...,...,...,...,...
2662743,1095491695,8200,water,NaN,11.902024,45.810205,nature
2662744,1095491696,8200,water,NaN,11.905475,45.812606,nature
2662745,1095603673,8200,water,NaN,11.797104,45.924658,nature
2662746,1095603675,8200,water,NaN,11.789369,45.927601,nature


In [64]:
n_rows, n_columns = 100, 100

lat_min, lat_max = df.lat.min(), df.lat.max()  # rows
lon_min, lon_max = df.lon.min(), df.lon.max()  # columns

diff_lat = lat_max - lat_min
diff_lon = lon_max - lon_min

step_lat = diff_lat / n_rows
step_lon = diff_lon / n_columns

In [65]:
categories = df['SELECT'].drop_duplicates().to_list()

In [66]:
starting_point_lat, starting_point_lon = lat_min, lon_max # start from the highest column and the first column

result_log = {}

#starting bbox
bbox_lat_min, bbox_lat_max = starting_point_lat, starting_point_lat + step_lat
bbox_lon_min, bbox_lon_max = starting_point_lon - step_lon, starting_point_lon

for row in range(n_rows):
    # reset starting point of the column axis
    bbox_lon_max = starting_point_lon
    bbox_lon_min = starting_point_lon - step_lon

    row_df = df.loc[(df.lat > bbox_lat_min) & (df.lat <= bbox_lat_max), :]

    for column in range(n_columns):
        cell_df = row_df.loc[(row_df.lon > bbox_lon_min) & (row_df.lon <= bbox_lon_max), :]

        cell_name = 'R' + str(row + 1) + 'C' + str(column + 1)
        cell_values = [row + 1, column +1, bbox_lat_min, bbox_lat_max, bbox_lon_min, bbox_lon_max]

        for category in categories:
            try:
                count = cell_df.SELECT.value_counts()[category]
                cell_values.append(count)

            except:
                cell_values.append(0)

        result_log[cell_name] = cell_values

        #print('LAT:', bbox_lat_min, '   ', bbox_lat_max, '  LON:', bbox_lon_min, '   ', bbox_lon_max, '    COUNT:', cell_df.shape[0])

        bbox_lon_max = bbox_lon_min
        bbox_lon_min -= step_lon

    bbox_lat_min = bbox_lat_max
    bbox_lat_max += step_lat

In [67]:
result_df = pd.DataFrame(result_log).T
result_df.columns = ['row', 'column', 'lat_min', 'lat_max', 'lon_min', 'lon_max'] + categories
result_df

,row,column,lat_min,lat_max,lon_min,lon_max,nature,entartainment,transports,art,sport,tourism,security,accessibility
R1C1,1.0,1.0,45.000004,45.010004,12.985000,13.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R1C2,1.0,2.0,45.000004,45.010004,12.970000,12.985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R1C3,1.0,3.0,45.000004,45.010004,12.955000,12.970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R1C4,1.0,4.0,45.000004,45.010004,12.940000,12.955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R1C5,1.0,5.0,45.000004,45.010004,12.925000,12.940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R100C96,100.0,96.0,45.989998,45.999998,11.560000,11.575,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
R100C97,100.0,97.0,45.989998,45.999998,11.545000,11.560,17.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
R100C98,100.0,98.0,45.989998,45.999998,11.530000,11.545,182.0,0.0,12.0,0.0,0.0,3.0,0.0,0.0
R100C99,100.0,99.0,45.989998,45.999998,11.515000,11.530,171.0,0.0,22.0,1.0,0.0,6.0,0.0,0.0


In [69]:
filtered_df = result_df[result_df[categories].T.sum(0) != 0] # remove the cells that have 0 on every fclass
filtered_df

,row,column,lat_min,lat_max,lon_min,lon_max,nature,entartainment,transports,art,sport,tourism,security,accessibility
R1C36,1.0,36.0,45.000004,45.010004,12.460000,12.475,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R1C37,1.0,37.0,45.000004,45.010004,12.445000,12.460,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R1C38,1.0,38.0,45.000004,45.010004,12.430000,12.445,54.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
R1C39,1.0,39.0,45.000004,45.010004,12.415000,12.430,46.0,1.0,13.0,1.0,0.0,0.0,0.0,0.0
R1C40,1.0,40.0,45.000004,45.010004,12.400000,12.415,17.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R100C96,100.0,96.0,45.989998,45.999998,11.560000,11.575,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
R100C97,100.0,97.0,45.989998,45.999998,11.545000,11.560,17.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
R100C98,100.0,98.0,45.989998,45.999998,11.530000,11.545,182.0,0.0,12.0,0.0,0.0,3.0,0.0,0.0
R100C99,100.0,99.0,45.989998,45.999998,11.515000,11.530,171.0,0.0,22.0,1.0,0.0,6.0,0.0,0.0


In [70]:
filtered_df.to_csv('1. filtered_df.csv')